In [ ]:
import sympy as sp
import numpy as np
import matplotlib.pyplot as plt
sp.init_printing()

## Next step identification

$$
\boldsymbol{\mathcal{R}}(\Delta{\boldsymbol{\mathcal{A}}}) = 0
$$

$$
\boldsymbol{\mathcal{R}}(\Delta{\boldsymbol{\mathcal{A}}}_{k+1}) =
\boldsymbol{\mathcal{R}}(\Delta{\boldsymbol{\mathcal{A}}}_k) +
\left.
    \frac{\partial \boldsymbol{\mathcal{R}} 
}{\partial \Delta{\boldsymbol{\mathcal{A}}}} 
\right|_{\Delta{\boldsymbol{\mathcal{A}}}_k}
\delta(\Delta{\boldsymbol{\mathcal{A}}}) = 0
$$

\begin{align}
\left.
    \frac{\partial \boldsymbol{\mathcal{R}} 
}{\partial \Delta{\boldsymbol{\mathcal{A}}}} 
\right|_{\Delta{\boldsymbol{\mathcal{A}}}_k}
\delta (\Delta{\boldsymbol{\mathcal{A}}}) &= - 
\boldsymbol{\mathcal{R}}(\Delta{\boldsymbol{\mathcal{A}}}_k) \\
\Delta{\boldsymbol{\mathcal{A}}}_{k+1} &= 
\Delta{\boldsymbol{\mathcal{A}}}_k + 
\delta (\Delta{\boldsymbol{\mathcal{A}}})
\end{align}


## Example for a workshop

In [ ]:
x, t = sp.symbols('x t', real=True)
x = sp.Function('x')(t)
f = sp.Function('f')(x)
dot_x = x.diff(t)
f = x**2
get_f = sp.lambdify((x), f, modules=['numpy'])

In [ ]:
dot_x - f

In [ ]:
t_n = sp.symbols('t_n', real=True)
t_n1 = sp.symbols('t_n1', real=True)
delta_t = sp.symbols(r'\Delta{t}', real=True)
x_n = sp.symbols('x_n', real=True)
delta_x = sp.symbols(r'\Delta{x}', real=True)
x_n1 = x_n + delta_x
a = sp.symbols('a', real=True)
x_na = a * x_n1 + (1 - a) * x_n
delta_x = x_n1 - x_n
delta_x

In [ ]:
x_na

In [ ]:
x_t = x_n + delta_x / (delta_t) * t
R_f_ = f.subs(x, x_na).subs(a, 0.5) - dot_x.subs(x, x_t).doit()
dR_dx_f_ = R_f_.diff(delta_x)
R_f_, dR_dx_f_

In [ ]:
dR_dx_f_
get_R_dR_f_ = sp.lambdify((x_n, delta_x, delta_t), (R_f_, dR_dx_f_), 'numpy')

In [ ]:
x_val = 0
delta_t_val = 1
delta_x_val = 1
t_range = np.linspace(0, 10, 11)
x_range = np.zeros_like(t_range)
f_range = np.zeros_like(t_range)
for n in range(1, len(t_range)):
    t_val = t_range[n]
    x_range[n] = x_val
    f_range[n] = get_f(x_val)
    delta_x_val = 1
    delta_t_val = t_range[n] - t_range[n-1]
    for k in range(8):
        print('x', x_val)
        R_f_k, dR_dx_f_k = get_R_dR_f_(x_val, delta_x_val, delta_t_val)
        print('R', R_f_k)
        delta_x_val -= R_f_k / dR_dx_f_k
    print('delta_x', delta_x_val)
    x_val += delta_x_val
x_range, f_range

In [ ]:
delta_x_val

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import fsolve

# Differential equation parameters
t_max = 10.0
delta_t = 0.1
num_steps = int(t_max / delta_t)

# Initial condition
epsilon_initial = 0.1

# Time array
time = np.linspace(0, t_max, num_steps + 1)

# Array to store the solution
epsilon = np.zeros(num_steps + 1)
epsilon[0] = epsilon_initial

# Function to solve for epsilon at the next time step using backward Euler
def backward_euler_step(epsilon_current, delta_t):
    # Define the implicit equation for the backward Euler step
    func = lambda epsilon_next: epsilon_next - epsilon_current - delta_t * (1 - epsilon_next) * epsilon_next
    epsilon_next = fsolve(func, epsilon_current)
    return epsilon_next[0]

# Time-stepping loop
for n in range(num_steps):
    epsilon[n + 1] = backward_euler_step(epsilon[n], delta_t)

# Plotting the results
plt.figure(figsize=(10, 6))
plt.plot(time, epsilon, label='Backward Euler', marker='o')
plt.title('Solution to $\\dot{\\epsilon} = (1 - \\epsilon) \\cdot \\epsilon$ using Backward Euler')
plt.xlabel('Time')
plt.ylabel('$\\epsilon$')
plt.grid(True)
plt.legend()
plt.show()

1. **Define the Residual**: You have a residual vector $ R(Eps, dot\_Eps) = 0 $, representing the implicit ODEs.

2. **Discretize the Variables**: You have discretized the variables using:
   $$
   Eps = Eps_n + \Delta Eps, \quad dot\_Eps = \frac{\Delta Eps}{\Delta t}
   $$
   
3. **Substitute** these discrete representations into the residual.

4. **Linearize Using Taylor Expansion**: Using a first-order Taylor expansion around the known values, you arrive at the system:
   $$
   J \cdot \Delta Eps = -R
   $$
   where $ J = \frac{\partial R}{\partial Eps} $ is the Jacobian matrix.

5. **Solve the Linear System**: This linear system can be solved for $\Delta Eps$, updating the solution iteratively.

Let's demonstrate this with a simple example using Python and `sympy` for symbolic computation:

```python
import sympy as sp
import numpy as np
from scipy.linalg import solve

# Define symbols
eps_1, eps_2 = sp.symbols('eps_1 eps_2')
dot_eps_1, dot_eps_2, delta_t = sp.symbols('dot_eps_1 dot_eps_2 delta_t')
Eps_n1, Eps_n2 = sp.symbols('Eps_n1 Eps_n2')
delta_Eps1, delta_Eps2 = sp.symbols('delta_Eps1 delta_Eps2')

# Define the residual vector R(Eps, dot_Eps)
R1 = dot_eps_1 - (1 - eps_1) * eps_1 - eps_2
R2 = dot_eps_2 - eps_1 * eps_2
R = sp.Matrix([R1, R2])

# Substitute discretization into the residual
Eps = [eps_1, eps_2]
dot_Eps = [delta_Eps1/delta_t, delta_Eps2/delta_t]
R_sub = R.subs({eps_1: Eps_n1 + delta_Eps1, eps_2: Eps_n2 + delta_Eps2,
                dot_eps_1: delta_Eps1/delta_t, dot_eps_2: delta_Eps2/delta_t})

# Derive the Jacobian J = dR/dEps
J = R_sub.jacobian([delta_Eps1, delta_Eps2])

# Create a function to evaluate R_sub and J
R_func = sp.lambdify((Eps_n1, Eps_n2, delta_Eps1, delta_Eps2, delta_t), R_sub, 'numpy')
J_func = sp.lambdify((Eps_n1, Eps_n2, delta_Eps1, delta_Eps2, delta_t), J, 'numpy')

# Solver function using Newton's method
def newton_solver(Eps_n, delta_t, tol=1e-6, max_iter=50):
    delta_Eps = np.zeros_like(Eps_n)
    for _ in range(max_iter):
        R_val = np.array(R_func(*Eps_n, *delta_Eps, delta_t)).astype(float).ravel()
        J_val = np.array(J_func(*Eps_n, *delta_Eps, delta_t)).astype(float)
        
        if np.linalg.norm(R_val) < tol:
            break

        delta_Eps -= np.linalg.solve(J_val, R_val)
    
    return Eps_n + delta_Eps

# Example usage
Eps_n = np.array([0.1, 0.1])  # Initial condition
delta_t = 0.1  # Time step

# Solving for the next time step
Eps_next = newton_solver(Eps_n, delta_t)
print("Eps_next:", Eps_next)
```

### Explanation:

- **SymPy Setup**: We first set up symbolic expressions using SymPy for the residuals.
- **Discretization Substitution**: We substitute $ Eps = Eps_n + \Delta Eps $ and $ dot\_Eps = \frac{\Delta Eps}{\Delta t} $ into the residuals.
- **Jacobian Calculation**: Use SymPy's Jacobian function to obtain the derivative matrix $ J $.
- **Numerical Function Conversion**: Convert the symbolic representations to numerical functions using `lambdify`.
- **Solver**: Implement Newton's method to solve for $ \Delta Eps $ iteratively.

This script gives you a starting framework to apply a Newton-Raphson based method for solving implicit ODE systems using backward Euler discretization. Adjust the `R1` and `R2` expressions to match your specific system.

In [ ]:
import sympy as sp
import numpy as np
from scipy.linalg import solve

# Define symbols
eps_1, eps_2 = sp.symbols('eps_1 eps_2')
dot_eps_1, dot_eps_2, delta_t = sp.symbols('dot_eps_1 dot_eps_2 delta_t')
Eps_n1, Eps_n2 = sp.symbols('Eps_n1 Eps_n2')
delta_Eps1, delta_Eps2 = sp.symbols('delta_Eps1 delta_Eps2')

# Define the residual vector R(Eps, dot_Eps)
R1 = dot_eps_1 - (1 - eps_1) * eps_1 - eps_2
R2 = dot_eps_2 - eps_1 * eps_2
R = sp.Matrix([R1, R2])

# Substitute discretization into the residual
Eps = [eps_1, eps_2]
dot_Eps = [delta_Eps1/delta_t, delta_Eps2/delta_t]
R_sub = R.subs({eps_1: Eps_n1 + delta_Eps1, eps_2: Eps_n2 + delta_Eps2,
                dot_eps_1: delta_Eps1/delta_t, dot_eps_2: delta_Eps2/delta_t})

# Derive the Jacobian J = dR/dEps
J = R_sub.jacobian([delta_Eps1, delta_Eps2])

# Create a function to evaluate R_sub and J
R_func = sp.lambdify((Eps_n1, Eps_n2, delta_Eps1, delta_Eps2, delta_t), R_sub, 'numpy')
J_func = sp.lambdify((Eps_n1, Eps_n2, delta_Eps1, delta_Eps2, delta_t), J, 'numpy')

# Solver function using Newton's method
def newton_solver(Eps_n, delta_t, tol=1e-6, max_iter=50):
    delta_Eps = np.zeros_like(Eps_n)
    for i in range(max_iter):
        R_val = np.array(R_func(*Eps_n, *delta_Eps, delta_t)).astype(float).ravel()
        print(i, R_val)
        if np.linalg.norm(R_val) < tol:
            break
        J_val = np.array(J_func(*Eps_n, *delta_Eps, delta_t)).astype(float)
        delta_Eps -= np.linalg.solve(J_val, R_val)
    
    return Eps_n + delta_Eps

# Example usage
Eps_n = np.array([0.1, 0.1])  # Initial condition
delta_t = 0.1  # Time step

# Solving for the next time step
Eps_next = newton_solver(Eps_n, delta_t)
print("Eps_next:", Eps_next)